<a href="https://colab.research.google.com/github/klipbn/klip_alex/blob/main/determination_average_number_lodsman_users/determination_average_number_lodsman_users.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Key ID:  
458744584072543631 - not actived 
719635943846940421 - ЛОЦМАН:КБ 2022,HSL,сетвр  
834732166184233058 - ОЭ КОМПАC-3D v20,HSL,сетвр  
982786152807025115 - ЛОЦМАН:КБ 2021,HSL,сетвр 

Productid:  
3197 ЛОЦМАН:КБ 2022,HSL,сетвр   
3258 КОМПАC-3D v20,HSL,сетвр 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Читаем данные с логами:

In [ ]:
def merge_logs(month, day):

  for i in range(1, day):

    # чтение файла
    df = pd.read_csv(f"/content/drive/My Drive/Projects/determination_average_number_lodsman_users/logs/2021_{month:02d}_{i:02d}_access.log", 
                    sep=r' ', skiprows=1, error_bad_lines=False, warn_bad_lines=False) 
    # задаем признаки
    df.columns = ['nan',	'nan1',	'date', 'time',	'ip',	'user',	'port',	'api' ,	'features' ,	'result' ,	'nan2']
    # уберем лишнии признаки
    df.drop(['nan', 'nan1', 'nan2'], axis=1, inplace=True)
    # удалим пропущенные значения
    df = df.dropna(subset=['user', 'features'])

    # проведем логическую индексацию
    mask = df["user"].str.contains(r"Администратор")
    dff = df.loc[mask]
    mask = dff["features"].str.contains(r"productid=3197")
    dff = dff.loc[mask]


  # соединим логи в одну таблицу
    if i == 1: 
      df_null = pd.DataFrame([np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]).T
      df_null.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]

      df_null = pd.concat([df_null, dff], ignore_index=True)
    else:

      df_null = pd.concat([df_null, dff], ignore_index=True)  

  # уберем лишние признаки и пропущенные значения из созданного переходного датафрейма
  df_null.drop(['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype'], axis=1, inplace=True) 
  df_null = df_null.dropna()
  return df_null

Сделаем из признака `'features'` датафрейм:

In [ ]:
def split_features(data):
  for i in range(len(data)-1):
    try:
      if i == 0: 
        zz555 = pd.DataFrame([np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]).T
        zz555.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
        zz = pd.DataFrame(str(data["features"][i:i+1]).split(',')).T
        zz.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
        zz2 = pd.DataFrame(str(data["features"][i+1:i+2]).split(',')).T
        zz2.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
        zz555 = pd.concat([zz555, zz, zz2], ignore_index=True)
      else:
        zz = pd.DataFrame(str(data["features"][i:i+1]).split(',')).T
        zz.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
        zz2 = pd.DataFrame(str(data["features"][i+1:i+2]).split(',')).T
        zz2.columns = ['median_time', 'haspid', 'productid', 'feat', 'sess', 'api', 'span', 'dtype' ]
        zz555 = pd.concat([zz555, zz, zz2], ignore_index=True)
    except: # если будет проблемная строка
      i += 1

  zzz5556 = zz555.drop_duplicates().dropna().reset_index()   # датафрейм из признака 
  return zzz5556

Соединим наши датафреймы с признаком `'features'` и с логами. Уберем лишние признаки, пропуски:

In [ ]:
def merge_logs_features(data1, data2):
  df = data1.merge(data2, how='outer', left_index=True, right_index=True)
  df.drop(['port', 'features', 'result', 'index', 'median_time', 'span', 'dtype'], axis=1, inplace=True)
  df = df.dropna().reset_index()
  return df

Прочитаем данные за октябрь:

In [ ]:
oct = merge_logs(10, 32)
oct_feat = split_features(oct)
df_oct = merge_logs_features(oct, oct_feat)

Прочитаем данные за ноябрь:

In [ ]:
nov = merge_logs(11, 31)
nov_feat = split_features(nov)
df_nov = merge_logs_features(nov, nov_feat)

Прочитаем данные за декабрь:

In [ ]:
dec = merge_logs(12, 32)
dec_feat = split_features(dec)
df_dec = merge_logs_features(dec, dec_feat)

Соединим все данные вместе:

In [ ]:
df = pd.concat([df_oct, df_nov, df_dec], ignore_index=True)

In [ ]:
df.tail()

,index,date,time,ip,user,haspid,productid,feat,sess,api
7404,361,0}2021-12-29,11:44:48,192.168.0.158:62833,Администратор@SRV-KB,haspid=719635943846940421,productid=3197,feat=622,sess=000001E9,api=7.50
7405,362,0}2021-12-29,11:59:07,192.168.0.158:62915,Администратор@SRV-KB,haspid=719635943846940421,productid=3197,feat=622,sess=000001E9,duration=7
7406,363,0}2021-12-29,14:17:39,192.168.0.158:63737,Администратор@SRV-KB,haspid=719635943846940421,productid=3197,feat=622,sess=000001DD,duration=12271
7407,364,0}2021-12-29,14:17:46,192.168.0.158:63737,Администратор@SRV-KB,haspid=719635943846940421,productid=3197,feat=622,sess=000001C1,duration=20529
7408,365,0}2021-12-29,14:50:16,192.168.0.158:63922,Администратор@SRV-KB,haspid=719635943846940421,productid=3197,feat=622,sess=000001C4,duration=27022


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7409 entries, 0 to 7408
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      7409 non-null   int64 
 1   date       7409 non-null   object
 2   time       7409 non-null   object
 3   ip         7409 non-null   object
 4   user       7409 non-null   object
 5   haspid     7409 non-null   object
 6   productid  7409 non-null   object
 7   feat       7409 non-null   object
 8   sess       7409 non-null   object
 9   api        7409 non-null   object
dtypes: int64(1), object(9)
memory usage: 579.0+ KB


In [ ]:
df['time'] = (pd.to_datetime(df['time'], format='%H:%M:%S')).dt.time

In [ ]:
df.describe()

,index
count,7409.000000
mean,1903.837225
std,1364.282426
min,1.000000
25%,744.000000
50%,1670.000000
75%,2947.000000
max,4799.000000
